In [3]:
import pandas as pd
import numpy as np

In [4]:
#import data into pandas dataframe

df = pd.read_csv('JC-202209-citibike-tripdata.csv')

In [5]:
# Describe dataframe - Good to start to get a sample idea of the numerical data colums.

df.describe()

,start_lat,start_lng,end_lat,end_lng
count,104247.000000,104247.000000,104068.000000,104068.000000
mean,40.732443,-74.040083,40.732428,-74.039875
std,0.012072,0.012098,0.012222,0.012237
min,40.706531,-74.086799,40.650000,-74.120000
25%,40.721525,-74.045953,40.721124,-74.045572
50%,40.735208,-74.037683,40.735208,-74.036904
75%,40.742258,-74.030970,40.742258,-74.030970
max,40.754992,-74.023738,40.849972,-73.931594


In [6]:
# Group data on count of routes

df_grouped = df.groupby([
       'start_station_name', 'end_station_name']
       )['started_at','ended_at'].agg('count')

df_grouped = df_grouped.reset_index()


# Remove routes where bike did not change stations

df_grouped = df_grouped[df_grouped['start_station_name'] != df_grouped['end_station_name']]

# cleanup

df_grouped['count'] = df_grouped['started_at']

df_grouped = df_grouped.drop(columns=['started_at','ended_at'])

df_grouped.to_csv('agreggated_counts_trips.csv')

/var/folders/p6/325r0tsd2cd1jddpt9nd70hm0000gn/T/ipykernel_10644/2143969827.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_grouped = df.groupby([


In [7]:
# Quick vis of frequency of counts per route - this is relevate to understand how many routes were taken only once or twice. 
# If the number is not sigficant we can exclude these to avoid extra computation for the MVP and call the API less times

import plotly

import plotly.express as px
df = df_grouped
fig = px.histogram(df, x="count")
fig.show()

In [8]:
# From the chart we can see that a huge amount of routes that were taken only onece (600)  - remove noisy routes 

df_grouped_light = df_grouped[df_grouped['count'] > 5]

In [9]:
# Create Metatada Dataset. All station names with the respective coordinates
# Each ride with have start and end coordinates, however, for the same station there will multiple corrdinate values that differ for a different in meters (minute)
# To avoid issues in the groupBy statements we average all coordinates values for each locations's latitute and longitude.

df_stations = pd.read_csv('JC-202209-citibike-tripdata.csv')[['start_station_name', 'end_station_name',  'start_lat', 'start_lng', 'end_lat', 'end_lng']]

df_stations_start = df_stations[['start_station_name','start_lat','start_lng']]

df_stations_end = df_stations[['end_station_name','end_lat','end_lng']]


# Averaging coordinate values for start locations and end locations

df_stations_start = df_stations_start.groupby(['start_station_name'])['start_lat','start_lng'].agg('mean')
df_stations_end = df_stations_end.groupby(['end_station_name'])['end_lat','end_lng'].agg('mean')

df_stations_start.rename(columns = {'start_lat':'lat','start_lng':'lng'},inplace=True)
df_stations_start.reset_index()

df_stations_end.rename(columns = {'end_lat':'lat','end_lng':'lng'},inplace=True)
df_stations_end.reset_index()

df_stations_metadata = pd.concat([df_stations_start,df_stations_end])

df_stations_metadata = df_stations_metadata.reset_index()

# If there are duplicated entries for start and end stations (there will be), we drop one to have just one calculated coordinates field per stations.
# Doesnt matter which one we drop because coordinates in the duplicated pair will be fairly consistent.

df_stations_metadata = df_stations_metadata.drop_duplicates(subset='index')

df_stations_metadata.to_csv('stations_coordinates.csv')

/var/folders/p6/325r0tsd2cd1jddpt9nd70hm0000gn/T/ipykernel_10644/1462100445.py:14: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

/var/folders/p6/325r0tsd2cd1jddpt9nd70hm0000gn/T/ipykernel_10644/1462100445.py:15: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [10]:
df_stations_metadata

,index,lat,lng
0,11 St & Washington St,40.749990,-74.027187
1,12 St & Sinatra Dr N,40.750615,-74.024057
2,14 St Ferry - 14 St & Shipyard Ln,40.752916,-74.024312
3,4 St & Grand St,40.742268,-74.035083
4,5 Corners Library,40.734974,-74.059526
...,...,...,...
229,Water St & Gouverneur Ln,40.704697,-74.008034
230,West St & Chambers St,40.717548,-74.013221
231,West Thames St,40.708347,-74.017134
232,William St & Pine St,40.707179,-74.008873


In [12]:
# Create Final Dataset containing for each rout the count of rides that stated and ended in that routes path and the coordinates 
# for begining and end locations


df_1 = df_grouped.merge(df_stations_metadata,how='right', left_on='start_station_name',right_on='index')
df_2 = df_1.merge(df_stations_metadata,how='right', left_on='end_station_name',right_on='index')


In [13]:
df_2.to_csv('final.csv')

In [24]:
df_2

,start_station_name,end_station_name,count,index_x,lat_x,lng_x,index_y,lat_y,lng_y
0,12 St & Sinatra Dr N,11 St & Washington St,31.0,12 St & Sinatra Dr N,40.750615,-74.024057,11 St & Washington St,40.749990,-74.027187
1,14 St Ferry - 14 St & Shipyard Ln,11 St & Washington St,46.0,14 St Ferry - 14 St & Shipyard Ln,40.752916,-74.024312,11 St & Washington St,40.749990,-74.027187
2,4 St & Grand St,11 St & Washington St,28.0,4 St & Grand St,40.742268,-74.035083,11 St & Washington St,40.749990,-74.027187
3,5 Corners Library,11 St & Washington St,1.0,5 Corners Library,40.734974,-74.059526,11 St & Washington St,40.749990,-74.027187
4,6 St & Grand St,11 St & Washington St,15.0,6 St & Grand St,40.744379,-74.034478,11 St & Washington St,40.749990,-74.027187
...,...,...,...,...,...,...,...,...,...
5223,Van Vorst Park,West St & Chambers St,1.0,Van Vorst Park,40.718479,-74.047733,West St & Chambers St,40.717548,-74.013221
5224,Morris Canal,West Thames St,2.0,Morris Canal,40.712385,-74.038468,West Thames St,40.708347,-74.017134
5225,Washington St,West Thames St,4.0,Washington St,40.724292,-74.035481,West Thames St,40.708347,-74.017134
5226,Harborside,William St & Pine St,1.0,Harborside,40.719259,-74.034230,William St & Pine St,40.707179,-74.008873


In [21]:
df_2.columns

Index(['start_station_name', 'end_station_name', 'count', 'index_x', 'lat_x',
       'lng_x', 'index_y', 'lat_y', 'lng_y'],
      dtype='object')